# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 10:20:15.338000 3828198 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:20:15.338000 3828198 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:20:17] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38074, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=25442

[2025-08-11 10:20:18] Using default HuggingFace chat template with detected content format: string


W0811 10:20:25.152000 3829147 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:20:25.152000 3829147 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0811 10:20:25.917000 3829148 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:20:25.917000 3829148 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=32.50 GB): 100%|██████████| 4/4 [00:09<00:00,  2.45s/it]
[2025-08-11 10:20:45] Capture draft cuda graph end. Time elapsed: 10.34 s. mem usage=4.06 GB. avail mem=28.71 GB.
[2025-08-11 10:20:45] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=28.71 GB


Capturing batches (bs=1 avail_mem=28.53 GB): 100%|██████████| 4/4 [00:00<00:00, 76.17it/s]
[2025-08-11 10:20:46] Capture draft extend cuda graph end. Time elapsed: 0.66 s. mem usage=0.21 GB. avail mem=28.50 GB.
[2025-08-11 10:20:46] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=28.50 GB
[2025-08-11 10:20:46] INFO:     Started server process [3828198]
[2025-08-11 10:20:46] INFO:     Waiting for application startup.
[2025-08-11 10:20:46] INFO:     Application startup complete.
[2025-08-11 10:20:46] INFO:     Uvicorn running on http://127.0.0.1:38074 (Press CTRL+C to quit)


[2025-08-11 10:20:47] INFO:     127.0.0.1:33774 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 10:20:47] INFO:     127.0.0.1:33786 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 10:20:47] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 10:20:47] INFO:     127.0.0.1:33792 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 10:20:47] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 10:20:52] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 10:20:52] INFO:     127.0.0.1:53748 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 10:20:58.821000 3832371 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:20:58.821000 3832371 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:21:01] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32885, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1849836

[2025-08-11 10:21:01] Using default HuggingFace chat template with detected content format: string


W0811 10:21:10.475000 3832898 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:21:10.475000 3832898 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0811 10:21:10.896000 3832899 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:21:10.896000 3832899 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=36.25 GB): 100%|██████████| 4/4 [00:18<00:00,  4.52s/it]
[2025-08-11 10:22:20] Capture draft cuda graph end. Time elapsed: 18.84 s. mem usage=0.74 GB. avail mem=35.69 GB.
[2025-08-11 10:22:20] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=35.69 GB


Capturing batches (bs=1 avail_mem=45.49 GB): 100%|██████████| 4/4 [00:00<00:00, 71.87it/s]
[2025-08-11 10:22:21] Capture draft extend cuda graph end. Time elapsed: 1.29 s. mem usage=-9.80 GB. avail mem=45.49 GB.
[2025-08-11 10:22:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=45.49 GB
[2025-08-11 10:22:22] INFO:     Started server process [3832371]
[2025-08-11 10:22:22] INFO:     Waiting for application startup.
[2025-08-11 10:22:22] INFO:     Application startup complete.
[2025-08-11 10:22:22] INFO:     Uvicorn running on http://127.0.0.1:32885 (Press CTRL+C to quit)


[2025-08-11 10:22:22] INFO:     127.0.0.1:35056 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 10:22:23] INFO:     127.0.0.1:35066 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 10:22:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 10:22:25] INFO:     127.0.0.1:35074 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 10:22:25] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 10:22:27] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 10:22:27] INFO:     127.0.0.1:35076 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 10:22:34.497000 3842238 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:22:34.497000 3842238 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:22:35] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35897, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-11 10:22:36] Using default HuggingFace chat template with detected content format: string


W0811 10:22:43.207000 3842885 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:22:43.207000 3842885 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 10:22:43.209000 3842884 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:22:43.209000 3842884 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:22:44] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=54.11 GB): 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]
[2025-08-11 10:23:14] Capture draft cuda graph end. Time elapsed: 7.31 s. mem usage=4.71 GB. avail mem=53.85 GB.
[2025-08-11 10:23:14] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.85 GB


Capturing batches (bs=1 avail_mem=56.01 GB): 100%|██████████| 4/4 [00:00<00:00, 90.43it/s]
[2025-08-11 10:23:14] Capture draft extend cuda graph end. Time elapsed: 0.61 s. mem usage=-2.16 GB. avail mem=56.01 GB.
[2025-08-11 10:23:14] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=56.01 GB


[2025-08-11 10:23:15] INFO:     Started server process [3842238]
[2025-08-11 10:23:15] INFO:     Waiting for application startup.
[2025-08-11 10:23:15] INFO:     Application startup complete.
[2025-08-11 10:23:15] INFO:     Uvicorn running on http://127.0.0.1:35897 (Press CTRL+C to quit)


[2025-08-11 10:23:16] INFO:     127.0.0.1:42628 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-11 10:23:16] INFO:     127.0.0.1:42640 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 10:23:16] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 10:23:16] INFO:     127.0.0.1:42648 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 10:23:16] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 10:23:21] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 10:23:21] INFO:     127.0.0.1:42658 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 10:23:28.027000 3846157 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:23:28.027000 3846157 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:23:29] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34983, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-11 10:23:29] Using default HuggingFace chat template with detected content format: string


W0811 10:23:36.321000 3847011 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:23:36.321000 3847011 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 10:23:36.321000 3847010 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:23:36.321000 3847010 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:23:37] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.24 GB):  75%|███████▌  | 3/4 [00:03<00:00,  1.35it/s]

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:05<00:00,  1.27s/it]
[2025-08-11 10:24:05] Capture draft cuda graph end. Time elapsed: 5.75 s. mem usage=-0.14 GB. avail mem=58.20 GB.
[2025-08-11 10:24:05] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.20 GB


Capturing batches (bs=1 avail_mem=58.09 GB): 100%|██████████| 4/4 [00:00<00:00, 93.79it/s]
[2025-08-11 10:24:05] Capture draft extend cuda graph end. Time elapsed: 0.75 s. mem usage=0.11 GB. avail mem=58.09 GB.
[2025-08-11 10:24:05] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.09 GB


[2025-08-11 10:24:06] INFO:     Started server process [3846157]
[2025-08-11 10:24:06] INFO:     Waiting for application startup.
[2025-08-11 10:24:06] INFO:     Application startup complete.
[2025-08-11 10:24:06] INFO:     Uvicorn running on http://127.0.0.1:34983 (Press CTRL+C to quit)
[2025-08-11 10:24:06] INFO:     127.0.0.1:59764 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 10:24:07] INFO:     127.0.0.1:59772 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 10:24:07] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 10:24:07] INFO:     127.0.0.1:59776 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 10:24:07] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 10:24:11] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 10:24:11] INFO:     127.0.0.1:52942 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 10:24:18.276000 3850694 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:24:18.276000 3850694 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:24:19] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=34625, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=762018103, constrained_jso

[2025-08-11 10:24:20] Using default HuggingFace chat template with detected content format: string


W0811 10:24:27.540000 3851563 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:24:27.540000 3851563 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 10:24:27.540000 3851564 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 10:24:27.540000 3851564 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 10:24:29] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-11 10:24:29] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=43.39 GB):  50%|█████     | 2/4 [00:01<00:01,  1.34it/s]

Capturing batches (bs=1 avail_mem=43.39 GB): 100%|██████████| 4/4 [00:02<00:00,  1.74it/s]
[2025-08-11 10:24:43] Capture draft cuda graph end. Time elapsed: 2.97 s. mem usage=3.34 GB. avail mem=41.48 GB.
[2025-08-11 10:24:43] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=41.48 GB


Capturing batches (bs=1 avail_mem=41.39 GB): 100%|██████████| 4/4 [00:00<00:00, 21.35it/s]
[2025-08-11 10:24:44] Capture draft extend cuda graph end. Time elapsed: 0.86 s. mem usage=0.09 GB. avail mem=41.39 GB.
[2025-08-11 10:24:44] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=41.39 GB


[2025-08-11 10:24:45] INFO:     Started server process [3850694]
[2025-08-11 10:24:45] INFO:     Waiting for application startup.
[2025-08-11 10:24:45] INFO:     Application startup complete.
[2025-08-11 10:24:45] INFO:     Uvicorn running on http://0.0.0.0:34625 (Press CTRL+C to quit)


[2025-08-11 10:24:46] INFO:     127.0.0.1:47278 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-11 10:24:46] INFO:     127.0.0.1:47290 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 10:24:46] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 10:24:46] INFO:     127.0.0.1:47296 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 10:24:46] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-11 10:24:51] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 10:24:51] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: True, gen throughput (token/s): 10.26, #queue-req: 0, 


[2025-08-11 10:24:51] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 151.41, #queue-req: 0, 


[2025-08-11 10:24:52] Decode batch. #running-req: 1, #token: 228, token usage: 0.01, accept len: 1.75, cuda graph: True, gen throughput (token/s): 153.29, #queue-req: 0, 


[2025-08-11 10:24:52] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, accept len: 1.60, cuda graph: True, gen throughput (token/s): 141.98, #queue-req: 0, 


[2025-08-11 10:24:53] Decode batch. #running-req: 1, #token: 354, token usage: 0.02, accept len: 1.55, cuda graph: True, gen throughput (token/s): 141.09, #queue-req: 0, 


[2025-08-11 10:24:53] Decode batch. #running-req: 1, #token: 420, token usage: 0.02, accept len: 1.65, cuda graph: True, gen throughput (token/s): 145.67, #queue-req: 0, 


[2025-08-11 10:24:54] Decode batch. #running-req: 1, #token: 484, token usage: 0.02, accept len: 1.60, cuda graph: True, gen throughput (token/s): 140.10, #queue-req: 0, 


[2025-08-11 10:24:54] Decode batch. #running-req: 1, #token: 553, token usage: 0.03, accept len: 1.73, cuda graph: True, gen throughput (token/s): 156.44, #queue-req: 0, 
[2025-08-11 10:24:54] INFO:     127.0.0.1:47302 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).